In [1]:
# Initial setup by importing needed modules
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
test = pd.read_csv('data/LoanStats3b.csv', header=1, low_memory=False)

In [3]:
# repeat all the data wrangling steps
test_drop = test.dropna(axis=1, how='all', thresh=40000).dropna(subset=['loan_amnt'])
test_drop.select_dtypes(include='float').fillna(0);
test_drop = test_drop.assign(loan_start_d=pd.to_datetime(test_drop.issue_d, format='%b-%Y'))
test_drop['yr_emp'] = test_drop['emp_length'].str[0:2].replace('< ',0).astype('float')
test_drop.earliest_cr_line = pd.to_datetime(test_drop.earliest_cr_line, format='%b-%Y')
test_drop['yr_credit']= (2013-test_drop.earliest_cr_line.dt.year).fillna(0)
test_drop['revol_util_dec'] = test_drop['revol_util'].str.replace(r'%',r'0').astype('float')/100
loan_dict = {'Fully Paid': 1, 'Charged Off': 0, 'Does not meet the credit policy. Status:Fully Paid': 1, 'Does not meet the credit policy. Status:Charged Off': 0 }
test_drop = test_drop.assign(target=test_drop.loan_status.map(loan_dict))

In [4]:
# capturing the needed features for machine learning validations
test_ml = test_drop[['purpose', 'yr_credit', 'dti', 'revol_util_dec', 'total_acc', 'addr_state', 'target']]

In [5]:
# data transformation to match training data setting
purpose = pd.get_dummies(test_ml['purpose'], drop_first=True)
print(test_ml)

                   purpose  yr_credit    dti  revol_util_dec  total_acc  \
0       debt_consolidation         24  15.55           0.444       22.0   
1       debt_consolidation         15  16.73           0.545       41.0   
2       debt_consolidation         22  15.75           0.346       16.0   
3       debt_consolidation         19  18.55           0.546       31.0   
4       debt_consolidation         14  27.06           0.709       17.0   
5         home_improvement         18   2.49           0.161        8.0   
6       debt_consolidation         27  22.87           0.612       27.0   
7       debt_consolidation          3   8.40           7.200        5.0   
8       debt_consolidation         19  14.05           0.216       22.0   
9       debt_consolidation         16  13.03           6.700       19.0   
10      debt_consolidation          4   4.62           2.400       18.0   
11             credit_card         24  22.18           0.767       39.0   
12      debt_consolidatio

In [6]:
# SML = pd.read_csv('data/SML.csv', header=1)
# SML_1 = SML.iloc[:,0].values.tolist()
# test_ml['helper_col'] = test_ml['addr_state'].isin(SML) #this serves as helper column only and will be drop later

In [7]:
# # test_ml['state'] = np.where(test_ml.helper_col == 1, 'SML', test_ml.addr_state)
# state = pd.get_dummies(test_ml['state'], drop_first = True)
# test_ml = pd.concat([lend, state], axis=1)

In [8]:
# export to a csv file 
test_ml.to_csv('data/lending_test.csv', index=False)